In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('test').getOrCreate()

23/08/05 17:30:45 WARN Utils: Your hostname, tns-des180 resolves to a loopback address: 127.0.1.1; using 172.24.31.35 instead (on interface enp2s0)
23/08/05 17:30:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/05 17:30:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df = spark.read.csv('./simpleplay.csv', inferSchema=True, header=True)

In [5]:
df.count()

610

In [6]:
df.show()

+---+---+------+-------------------+-------------------+--------+-------+------+-------+-----+---+----------+-------+-------------+-------+---------+--------+----------+--------------+-----+-------------------+-------------------+
|_c0| id|bet_id|           bet_time|        payout_time|username|host_id|detail|game_id|round|set|bet_amount|rolling|result_amount|balance|game_type|bet_type|bet_source|transaction_id|state|          create_at|          update_at|
+---+---+------+-------------------+-------------------+--------+-------+------+-------+-----+---+----------+-------+-------------+-------+---------+--------+----------+--------------+-----+-------------------+-------------------+
|  0|  1| 271.0|2008-11-14 18:05:56|1973-11-12 14:20:02|   kwamg|    530| xqogh|  idwjo|  189|808|       699|    218|          915|    865|    egqow|     843|       854|            55| true|2003-09-07 07:34:24|1983-03-29 19:39:41|
|  1|  6| 574.0|1990-09-20 17:46:16|2003-06-17 10:17:01|   nkrwj|     63| ep

23/08/05 17:30:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, bet_id, bet_time, payout_time, username, host_id, detail, game_id, round, set, bet_amount, rolling, result_amount, balance, game_type, bet_type, bet_source, transaction_id, state, create_at, update_at
 Schema: _c0, id, bet_id, bet_time, payout_time, username, host_id, detail, game_id, round, set, bet_amount, rolling, result_amount, balance, game_type, bet_type, bet_source, transaction_id, state, create_at, update_at
Expected: _c0 but found: 
CSV file: file:///home/user/Desktop/airflow/analysis/simpleplay.csv


In [7]:
from datetime import date
from pyspark.sql import Row
from pyspark.sql.functions import lit

In [8]:
spark = SparkSession.builder \
    .appName("HadoopSparkTest") \
    .master("spark://0f1d62354cc5:7077") \
    .enableHiveSupport() \
    .getOrCreate()

spark

23/08/05 17:30:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
today = date.today()
year = today.year
month = today.month
day = today.day

record = Row("key", "value")
df = spark.createDataFrame([record(i, "val" + str(i)) for i in range(1, 10)])
df.createOrReplaceTempView("records")

out_df = df.withColumn('year', lit(year)).withColumn('month', lit(month)).withColumn('day', lit(day))
out_df.show()

+---+-----+----+-----+---+
|key|value|year|month|day|
+---+-----+----+-----+---+
|  1| val1|2023|    8|  5|
|  2| val2|2023|    8|  5|
|  3| val3|2023|    8|  5|
|  4| val4|2023|    8|  5|
|  5| val5|2023|    8|  5|
|  6| val6|2023|    8|  5|
|  7| val7|2023|    8|  5|
|  8| val8|2023|    8|  5|
|  9| val9|2023|    8|  5|
+---+-----+----+-----+---+



In [10]:
tblLocation = 'hdfs://172.27.0.2:10000/user/hive/datalake/records'
out_df.write.partitionBy('year', 'month', 'day').mode('append').parquet(tblLocation)

Py4JJavaError: An error occurred while calling o66.parquet.
: java.net.ConnectException: Call From tns-des180/127.0.1.1 to 172.27.0.2:10000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:828)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1616)
	at org.apache.hadoop.ipc.Client.call(Client.java:1558)
	at org.apache.hadoop.ipc.Client.call(Client.java:1455)
	at org.apache.hadoop.ipc.ProtobufRpcEngine2$Invok